# Metadata Extraction and Analysis

## Imports and dataloading

In [2]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from ieeg.auth import Session
from scipy import signal as sig
import os
from os.path import join as ospj
from os.path import exists as ospe
import pathlib
from tqdm import tqdm

# Statistical imports
from scipy.stats import chi2_contingency, fisher_exact
from scipy.stats.contingency import association
from statsmodels.stats.proportion import proportions_ztest
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import statsmodels.formula.api as smf

from utils import *

In [3]:
import sys
sys.path.append('/users/wojemann/iEEG_processing')
from pioneer import Pioneer

In [4]:
usr,passpath,datapath,prodatapath,metapath,figpath,patient_table,rid_hup,pt_list = \
load_config(ospj('/mnt/leif/littlab/users/wojemann/stim-seizures/code','config.json'),'CHOP')

In [5]:
patient_table

,ptID,ieeg_ids,lf_stim,hf_stim,typical,mtle,focality,laterality,interictal_training
0,CHOP005,"[CHOPCCEP_005, CHOP005]",1,0,NaN,NaN,NaN,NaN,"[CHOP005, 14190.17]"
1,CHOP010,"[CHOPCCEP_010, CHOP010a, CHOP010b, CHOP010c]",1,0,NaN,NaN,NaN,NaN,[]
2,CHOP024,"[CHOPCCEP_024, CHOP024]",1,0,NaN,NaN,NaN,NaN,"[CHOP024, 112138.27]"
3,CHOP026,"[CHOPCCEP_026, CHOP026]",1,0,NaN,NaN,NaN,NaN,"[CHOP026, 76411.33]"
4,CHOP028,"[CHOPCCEP_028, CHOP028]",1,0,NaN,NaN,NaN,NaN,"[CHOP028, 7517.56]"
5,CHOP035,"[CHOPCCEP_035, CHOP035]",1,0,NaN,NaN,NaN,NaN,"[CHOP035, 82282.0]"
6,CHOP037,"[CHOPCCEP_037, CHOP037]",1,0,NaN,NaN,NaN,NaN,"[CHOP037, 58173.01]"
7,CHOP038,"[CHOPCCEP_038, CHOP038]",1,0,NaN,NaN,NaN,NaN,[]
8,CHOP041,"[CHOPCCEP_041, CHOP041]",1,1,NaN,NaN,NaN,NaN,"[CHOP041, 112959.7]"
9,CHOP044,"[CHOPCCEP_044, CHOP044]",1,0,NaN,NaN,NaN,NaN,"[CHOP044, 4070.79]"


## Creating annotation assignments

In [9]:
seizures_df = pd.read_csv(ospj(metapath,'stim_seizure_information - LF_seizure_annotation.csv'))
seizures_df = seizures_df[~seizures_df.Patient.isin(["HUP235","HUP238","HUP246","HUP261"])]
seizures_df = seizures_df[seizures_df['to_annotate']==1]
seizures_df['annotators'] = ""
seizures_df['approximate_onset'].fillna(seizures_df['UEO'],inplace=True)
seizures_df['approximate_onset'].fillna(seizures_df['EEC'],inplace=True)
seizures_df['approximate_onset'].fillna(seizures_df['Other_onset_description'],inplace=True)
seizures_df = seizures_df.drop(['to_annotate','Notes','source','EEC onset channels','UEO onset channels','EEC','UEO','Other_onset_description'],axis=1).reset_index(drop=True)
seizures_df.head()

,Patient,IEEGname,approximate_onset,end,stim,stim_channels,typical,LVFA,Summaries,annotators
0,HUP224,HUP224_phaseII,71156.5900,71190.99,1.0,LB1-LB2,0.0,0.0,456.0,
1,HUP224,HUP224_phaseII,339143.6435,339234.20,0.0,NaN,NaN,NaN,NaN,
2,HUP224,HUP224_phaseII,491467.8046,491541.43,0.0,NaN,NaN,NaN,NaN,
3,HUP224,HUP224_phaseII,519177.9500,519258.16,0.0,NaN,NaN,NaN,NaN,
4,HUP225,HUP225_phaseII,159834.1400,159913.05,1.0,RC1-RC2,1.0,0.0,NaN,


In [6]:
# Assuming you have a DataFrame named 'seizures_df' containing seizure data
# And a list of annotators initials 2,5
np.random.seed(10)
annotators = ['CK','EC','DZ','JJ','JK']
annotation_counts = {key: 0 for key in annotators}
def calc_weights(annotation_counts):
    weights = [1/(1+value) for value in annotation_counts.values()]
    tot_weight = sum(weights)
    return [w/tot_weight for w in weights]

# Assuming 'seizures_df' contains a column 'patient_id' indicating the patient ID for each seizure
# We'll first group seizures by patient_id
grouped_seizures = seizures_df.groupby('Patient')

# Dictionary to store DataFrames for each annotator
annotator_dfs = {}

# Create Primary DF that contains all seizures from all patients with all annotators per seizure
# Iterate over each patient group
for patient_id, patient_group in grouped_seizures:
    num_seizures = len(patient_group)
    # Randomly assign 3 annotators to the patient
    weights = calc_weights(annotation_counts)
    assigned_annotators = np.random.choice(annotators, size=3, replace=False,p=weights)
    annot_str = str(assigned_annotators)
    
    annotator_list = np.repeat(annot_str,num_seizures,0)
    # if len(annotator_list.shape) < 2:
    #     annotator_list = np.expand_dims(annotator_list,0)
    
    seizures_df.iloc[seizures_df.Patient == patient_id,-1] = annotator_list
    # Repeat the annotators for each seizure in the patient group
    for annotator in assigned_annotators:
        annotation_counts[annotator] += len(patient_group)
        if annotator in annotator_dfs.keys():
            annotator_dfs[annotator] = pd.concat([annotator_dfs[annotator],patient_group])
        else:
            annotator_dfs[annotator] = patient_group
print(annotation_counts)

{'CK': 34, 'EC': 42, 'DZ': 33, 'JJ': 35, 'JK': 36}


In [18]:
for key in annotator_dfs.keys():
    annotator_dfs[key][["UEO_time","UEO_ch","10sec_ch"]] = ""
    annotator_dfs[key].to_csv(ospj(prodatapath,f"stim_seizure_annotations_{key}.csv"),index=False)
seizures_df.to_csv(ospj(prodatapath,"LF_seizure_annotations_wannotator.csv"),index=False)

## Extracting seizure annotations from iEEG

In [13]:
# for i,pt in patient_table.iloc[[-2],:].iterrows():
for i,pt in patient_table[patient_table.ptID == 'CHOP032'].iterrows():
    for ieeg_pt in pt.ieeg_ids:
        # try:
            save_path = ospj(datapath,pt.ptID)
            print(ieeg_pt,save_path)
            if not ospe(save_path):
                os.makedirs(save_path)
            wagon = Pioneer(usr,passpath,ieeg_pt)
            wagon.pull_annotations()
            wagon.filter_seizure_annotations()
            wagon.seizure_annotations.to_csv(ospj(save_path,f'seizure_annotations_{ieeg_pt}.csv'))
        # except:
        #     continue

CHOPCCEP_032 /mnt/sauce/littlab/users/wojemann/stim-seizures/RAW_DATA/CHOP032
got 100 annotations on call # 1 covering 22925780761 usec to 25072323242 usec
got 100 annotations on call # 2 covering 25073326171 usec to 25969639160 usec
got 100 annotations on call # 3 covering 25971192871 usec to 26801372070 usec
got 100 annotations on call # 4 covering 26802388671 usec to 27668140625 usec
got 100 annotations on call # 5 covering 27673568847 usec to 28484145996 usec
got 100 annotations on call # 6 covering 28485167480 usec to 29377541015 usec
got 100 annotations on call # 7 covering 29377543945 usec to 30479178711 usec
got 100 annotations on call # 8 covering 30481079589 usec to 31405335937 usec
got 72 annotations on call # 9 covering 31520166015 usec to 33032063964 usec
Filtered       9.97% of all annotations
CHOP032 /mnt/sauce/littlab/users/wojemann/stim-seizures/RAW_DATA/CHOP032
got 100 annotations on call # 1 covering 49212890 usec to 32032358398 usec
got 100 annotations on call # 2 c

## Patient table

In [54]:
import pandas as pd
import numpy as np
import scipy.io
from scipy.stats import chi2_contingency, f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols

# --- Load metadata ---
stim_df = pd.read_csv(ospj(metapath,'stim_seizure_information - metadata-4.csv'))
chop_df = pd.read_csv(ospj(metapath,'CHOP_metadata.csv'))
# --- Format metadata ---
stim_df['sex'] = stim_df['sex'].map({1: 'M', 2: 'F'})
stim_df['mtle'] = stim_df['localization'].apply(lambda x: 1 if isinstance(x, str) and 'MTLE' in x else 0)
stim_df['lesional'] = stim_df['lesional'].apply(lambda x: 1 if x == 2 else 0)
stim_df['center'] = 'HUP'
stim_df['record_id'] = stim_df['record_id'].astype(str)
stim_df['hupsubjno'] = 'HUP' + stim_df['hupsubjno'].astype(int).astype(str)

chop_df['sex'] = chop_df['sex'].map({'M': 'M', 'F': 'F'})
chop_df['center'] = 'CHOP'
chop_df['record_id'] = chop_df['ptID'].astype(str)
chop_df['duration'] = chop_df['duration of epilepsy prior to stim (y)']
chop_df['age_at_onset'] = chop_df['age at epilepsy onset']

# --- Load and extract percent channels stimulated from .mat files ---
def extract_percent_stim(matfile, center):
    pts = matfile['pt'][0]
    pts_df = pd.DataFrame(pts)
    if center == 'HUP':
        pts_df['name'] = pts_df['name'].apply(lambda x: int(x[0][-3:]))
    else:
        pts_df['name'] = pts_df['name'].apply(lambda x: x[0])
    pts_df['nchs'] = pts_df['nchs'].apply(lambda x: x[0])
    pts_df['nstim'] = pts_df['nstim'].apply(lambda x: x[0])
    pts_df['percent_channels_stimulated'] = pts_df['nstim']/pts_df['nchs']*100
    pts_df['center'] = center    
    return pts_df[['name','nchs','nstim','percent_channels_stimulated','center']]

stim_info = scipy.io.loadmat(ospj(metapath,'stim_info.mat'))
stim_info_chop = scipy.io.loadmat(ospj(metapath,'stim_info_chop.mat'))
stim_stim = extract_percent_stim(stim_info, 'HUP')
stim_stim['name'] = 'HUP' + stim_stim.name.astype(str)
stim_chop = extract_percent_stim(stim_info_chop, 'CHOP')
stim_chop['name'] = stim_chop.name.apply(lambda x: ''.join(x.split('CCEP_')))
# --- Merge percent_channels_stimulated into metadata ---
def merge_percent_stim(meta_df, stim_df, id_col):
    # Use a fuzzy match if needed, here we use exact match for demonstration
    merged = meta_df.merge(stim_df[['name', 'percent_channels_stimulated']], how='left',
                           left_on=id_col, right_on='name')
    merged['percent_channels_stimulated'] = merged['percent_channels_stimulated'].astype(float)
    merged['patient'] = merged[id_col]
    return merged

stim_df = merge_percent_stim(stim_df, stim_stim, 'hupsubjno')
chop_df = merge_percent_stim(chop_df, stim_chop, 'ptID')
# chop_df.dropna(subset='mtle',inplace=True)
# stim_df.dropna(subset='mtle',inplace=True)
# --- Combine all data ---
stim_cols = ['patient', 'sex', 'age_at_onset', 'duration', 'lesional', 'mtle', 'unifocal', 'stim_sz', 'center', 'percent_channels_stimulated','outcome']
chop_cols = ['patient', 'sex', 'age_at_onset', 'duration', 'lesional', 'mtle', 'unifocal', 'stim_sz', 'center', 'percent_channels_stimulated','outcome']

combined_df = pd.concat([stim_df[stim_cols], chop_df[chop_cols]], ignore_index=True)

# --- Summary Table Construction ---
summary_rows = []

# Categorical variables
for var in ['sex', 'center', 'lesional', 'mtle', 'unifocal','outcome']:
    group_counts = combined_df.groupby(var)['stim_sz'].agg(['count', 'sum'])
    group_counts['percent'] = (group_counts['sum'] / group_counts['count'] * 100).round(1)
    contingency_table = pd.crosstab(combined_df[var], combined_df['stim_sz'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    n = contingency_table.values.sum()
    w = np.sqrt(chi2 / n)
    for group in group_counts.index:
        summary_rows.append({
            'Variable or Group': var,
            'Group': group,
            'Total Group': int(group_counts.loc[group, 'count']),
            'With Stim Seizure': int(group_counts.loc[group, 'sum']),
            'Response Rate, %': group_counts.loc[group, 'percent'],
            'P Value': round(p, 3),
            'Effect Size': round(w, 3)
        })

# Continuous variables
for var in ['age_at_onset', 'duration', 'percent_channels_stimulated']:
    for stim_val in [0, 1]:
        subset = combined_df[combined_df['stim_sz'] == stim_val][var].dropna()
        mean = subset.mean()
        std = subset.std()
        summary_rows.append({
            'Variable or Group': var,
            'Group': f"stim_sz={stim_val}",
            'Total Group': len(subset),
            'With Stim Seizure': round(mean, 2),
            'Response Rate, %': round(std, 2),
            'P Value': '',  # Will fill below
            'Effect Size': ''
        })
    # ANOVA and effect size
    groups = [combined_df[combined_df['stim_sz'] == val][var].dropna() for val in [0, 1]]
    if all(len(g) > 1 for g in groups):
        f_stat, p = f_oneway(*groups)
        model = ols(f'{var} ~ C(stim_sz)', data=combined_df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        eta_sq = anova_table['sum_sq'][0] / anova_table['sum_sq'].sum()
        # Fill in the last two rows (for stim_sz=0 and stim_sz=1)
        summary_rows[-2]['P Value'] = round(p, 3)
        summary_rows[-1]['P Value'] = round(p, 3)
        summary_rows[-2]['Effect Size'] = round(eta_sq, 3)
        summary_rows[-1]['Effect Size'] = round(eta_sq, 3)

summary_df = pd.DataFrame(summary_rows)

# --- Display the summary table ---
print(summary_df)


              Variable or Group      Group  Total Group  With Stim Seizure  \
0                           sex          F           54              20.00   
1                           sex          M           48              18.00   
2                        center       CHOP           50              17.00   
3                        center        HUP           55              21.00   
4                      lesional        0.0           52              22.00   
5                      lesional        1.0           50              16.00   
6                          mtle        0.0           61              15.00   
7                          mtle        1.0           38              21.00   
8                      unifocal        0.0           34              13.00   
9                      unifocal        1.0           65              23.00   
10                      outcome        0.0           18               8.00   
11                      outcome        1.0           27         

## Stim Sz and MTLE

### All patients

In [59]:
chi2_df = combined_df.dropna(subset=['mtle','stim_sz'])

In [ ]:
# Flip the table: rows = stim_sz, columns = mtle
contingency_rev = pd.crosstab(chi2_df['stim_sz'],chi2_df['mtle'])

# Chi-squared test
chi2_rev, p_rev, dof_rev, expected_rev = chi2_contingency(contingency_rev)

# Effect size
cramers_v_rev = association(contingency_rev.values)

df = chi2_df.copy()
# True: mtle (target), Predicted: stim_sz (used as predictor)
cm = confusion_matrix(df['mtle'], df['stim_sz'])  # rows = true mtle, cols = stim_sz

# Extract counts
TN, FP, FN, TP = cm.ravel()

# Compute PPV and NPV
ppv = TP / (TP + FP) if (TP + FP) else float('nan')
npv = TN / (TN + FN) if (TN + FN) else float('nan')

print(f"PPV (P(MTLE=1 | stim_sz=1)): {ppv:.3f}")
print(f"NPV (P(MTLE=0 | stim_sz=0)): {npv:.3f}")

print("Contingency Table (P(mtle | stim_sz)):\n", contingency_rev)
print(f"Chi2 = {chi2_rev:.4f}, p = {p_rev:.4f}, Cramer's V = {cramers_v_rev:.4f}")

PPV (P(MTLE=1 | stim_sz=1)): 0.583
NPV (P(MTLE=0 | stim_sz=0)): 0.730
Contingency Table (P(mtle | stim_sz)):
 mtle     0.0  1.0
stim_sz          
0         46   17
1         15   21
Chi2 = 8.2402, p = 0.0041, Cramer's V = 0.3101


In [61]:
model = smf.logit('mtle ~ stim_sz', data=combined_df).fit()
print(model.summary())
print(np.exp(model.params['stim_sz']))

Optimization terminated successfully.
         Current function value: 0.618044
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                   mtle   No. Observations:                   99
Model:                          Logit   Df Residuals:                       97
Method:                           MLE   Df Model:                            1
Date:                Fri, 09 May 2025   Pseudo R-squ.:                 0.07188
Time:                        12:36:56   Log-Likelihood:                -61.186
converged:                       True   LL-Null:                       -65.925
Covariance Type:            nonrobust   LLR p-value:                  0.002080
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9954      0.284     -3.507      0.000      -1.552      -0.439
stim_sz        1.3319      0.

### CHOP patients

In [71]:
df = chi2_df[chi2_df['patient'].apply(lambda x: 'CHOP' in x)]
print(len(df))
contingency_rev = pd.crosstab(df['stim_sz'],df['mtle'])
# Chi-squared test
chi2_rev, p_rev, dof_rev, expected_rev = chi2_contingency(contingency_rev)

# Effect size
cramers_v_rev = association(contingency_rev.values)
# True: mtle (target), Predicted: stim_sz (used as predictor)
cm = confusion_matrix(df['mtle'], df['stim_sz'])  # rows = true mtle, cols = stim_sz

# Extract counts
TN, FP, FN, TP = cm.ravel()

# Compute PPV and NPV
ppv = TP / (TP + FP) if (TP + FP) else float('nan')
npv = TN / (TN + FN) if (TN + FN) else float('nan')

print(f"PPV (P(MTLE=1 | stim_sz=1)): {ppv:.3f}")
print(f"NPV (P(MTLE=0 | stim_sz=0)): {npv:.3f}")
print("Contingency Table (P(mtle | stim_sz)):\n", contingency_rev)
print(f"Chi2 = {chi2_rev:.4f}, p = {p_rev:.4f}, Cramer's V = {cramers_v_rev:.4f}")
model = smf.logit('mtle ~ stim_sz', data=df).fit()
print(model.summary())
print(np.exp(model.params['stim_sz']))

44
PPV (P(MTLE=1 | stim_sz=1)): 0.333
NPV (P(MTLE=0 | stim_sz=0)): 0.931
Contingency Table (P(mtle | stim_sz)):
 mtle     0.0  1.0
stim_sz          
0         27    2
1         10    5
Chi2 = 3.3778, p = 0.0661, Cramer's V = 0.3426
Optimization terminated successfully.
         Current function value: 0.382395
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                   mtle   No. Observations:                   44
Model:                          Logit   Df Residuals:                       42
Method:                           MLE   Df Model:                            1
Date:                Fri, 09 May 2025   Pseudo R-squ.:                  0.1273
Time:                        12:41:29   Log-Likelihood:                -16.825
converged:                       True   LL-Null:                       -19.279
Covariance Type:            nonrobust   LLR p-value:                   0.02674
                 coef    std err  

### HUP patients

In [72]:
df = chi2_df[chi2_df.patient.apply(lambda x: 'HUP' in x)]
contingency_rev = pd.crosstab(df['stim_sz'],df['mtle'],)

# Chi-squared test
chi2_rev, p_rev, dof_rev, expected_rev = chi2_contingency(contingency_rev)

# Effect size
cramers_v_rev = association(contingency_rev.values)

# True: mtle (target), Predicted: stim_sz (used as predictor)
cm = confusion_matrix(df['mtle'], df['stim_sz'])  # rows = true mtle, cols = stim_sz

# Extract counts
TN, FP, FN, TP = cm.ravel()

# Compute PPV and NPV
ppv = TP / (TP + FP) if (TP + FP) else float('nan')
npv = TN / (TN + FN) if (TN + FN) else float('nan')

print(f"PPV (P(MTLE=1 | stim_sz=1)): {ppv:.3f}")
print(f"NPV (P(MTLE=0 | stim_sz=0)): {npv:.3f}")
print("Contingency Table (P(mtle | stim_sz)):\n", contingency_rev)
print(f"Chi2 = {chi2_rev:.4f}, p = {p_rev:.4f}, Cramer's V = {cramers_v_rev:.4f}")

model = smf.logit('mtle ~ stim_sz', data=df).fit()
print(model.summary())
print(np.exp(model.params['stim_sz']))

PPV (P(MTLE=1 | stim_sz=1)): 0.762
NPV (P(MTLE=0 | stim_sz=0)): 0.559
Contingency Table (P(mtle | stim_sz)):
 mtle     0.0  1.0
stim_sz          
0         19   15
1          5   16
Chi2 = 4.2038, p = 0.0403, Cramer's V = 0.3142
Optimization terminated successfully.
         Current function value: 0.633773
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                   mtle   No. Observations:                   55
Model:                          Logit   Df Residuals:                       53
Method:                           MLE   Df Model:                            1
Date:                Fri, 09 May 2025   Pseudo R-squ.:                 0.07482
Time:                        12:42:59   Log-Likelihood:                -34.858
converged:                       True   LL-Null:                       -37.676
Covariance Type:            nonrobust   LLR p-value:                   0.01758
                 coef    std err     

### Adult patients

In [83]:
df = chi2_df[chi2_df.age_at_onset.apply(lambda x: x > 14)]
contingency_rev = pd.crosstab(df['stim_sz'],df['mtle'])

# Chi-squared test
chi2_rev, p_rev, dof_rev, expected_rev = chi2_contingency(contingency_rev)

# Effect size
cramers_v_rev = association(contingency_rev.values)

# True: mtle (target), Predicted: stim_sz (used as predictor)
cm = confusion_matrix(df['mtle'], df['stim_sz'])  # rows = true mtle, cols = stim_sz

# Extract counts
TN, FP, FN, TP = cm.ravel()

# Compute PPV and NPV
ppv = TP / (TP + FP) if (TP + FP) else float('nan')
npv = TN / (TN + FN) if (TN + FN) else float('nan')

print(f"PPV (P(MTLE=1 | stim_sz=1)): {ppv:.3f}")
print(f"NPV (P(MTLE=0 | stim_sz=0)): {npv:.3f}")
print("Contingency Table (P(mtle | stim_sz)):\n", contingency_rev)
print(f"Chi2 = {chi2_rev:.4f}, p = {p_rev:.4f}, Cramer's V = {cramers_v_rev:.4f}")

model = smf.logit('mtle ~ stim_sz', data=df).fit()
print(model.summary())
print(np.exp(model.params['stim_sz']))

PPV (P(MTLE=1 | stim_sz=1)): 0.800
NPV (P(MTLE=0 | stim_sz=0)): 0.560
Contingency Table (P(mtle | stim_sz)):
 mtle     0.0  1.0
stim_sz          
0         14   11
1          3   12
Chi2 = 3.6078, p = 0.0575, Cramer's V = 0.3526
Optimization terminated successfully.
         Current function value: 0.616357
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                   mtle   No. Observations:                   40
Model:                          Logit   Df Residuals:                       38
Method:                           MLE   Df Model:                            1
Date:                Fri, 09 May 2025   Pseudo R-squ.:                 0.09606
Time:                        13:18:52   Log-Likelihood:                -24.654
converged:                       True   LL-Null:                       -27.274
Covariance Type:            nonrobust   LLR p-value:                   0.02208
                 coef    std err     

### Pediatric patients

In [84]:
df = chi2_df[chi2_df.age_at_onset.apply(lambda x: x <= 14)]
contingency_rev = pd.crosstab(df['stim_sz'],df['mtle'])

# Chi-squared test
chi2_rev, p_rev, dof_rev, expected_rev = chi2_contingency(contingency_rev)

# Effect size
cramers_v_rev = association(contingency_rev.values)

# True: mtle (target), Predicted: stim_sz (used as predictor)
cm = confusion_matrix(df['mtle'], df['stim_sz'])  # rows = true mtle, cols = stim_sz

# Extract counts
TN, FP, FN, TP = cm.ravel()

# Compute PPV and NPV
ppv = TP / (TP + FP) if (TP + FP) else float('nan')
npv = TN / (TN + FN) if (TN + FN) else float('nan')

print(f"PPV (P(MTLE=1 | stim_sz=1)): {ppv:.3f}")
print(f"NPV (P(MTLE=0 | stim_sz=0)): {npv:.3f}")
print("Contingency Table (P(mtle | stim_sz)):\n", contingency_rev)
print(f"Chi2 = {chi2_rev:.4f}, p = {p_rev:.4f}, Cramer's V = {cramers_v_rev:.4f}")

model = smf.logit('mtle ~ stim_sz', data=df).fit()
print(model.summary())
print(np.exp(model.params['stim_sz']))

PPV (P(MTLE=1 | stim_sz=1)): 0.450
NPV (P(MTLE=0 | stim_sz=0)): 0.865
Contingency Table (P(mtle | stim_sz)):
 mtle     0.0  1.0
stim_sz          
0         32    5
1         11    9
Chi2 = 5.3510, p = 0.0207, Cramer's V = 0.3491
Optimization terminated successfully.
         Current function value: 0.498526
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                   mtle   No. Observations:                   57
Model:                          Logit   Df Residuals:                       55
Method:                           MLE   Df Model:                            1
Date:                Fri, 09 May 2025   Pseudo R-squ.:                  0.1057
Time:                        13:21:34   Log-Likelihood:                -28.416
converged:                       True   LL-Null:                       -31.776
Covariance Type:            nonrobust   LLR p-value:                  0.009539
                 coef    std err     

## Typical and Outcome

In [ ]:
sz_metadata = pd.read_csv(ospj(metapath,'stim_seizure_information_BIDS.csv')).drop('Unnamed: 0',axis=1)

In [79]:
typical_df = sz_metadata.groupby('Patient').apply(lambda x: np.isin(1,x.typical).astype(int).item()).reset_index()
typical_df.rename(columns={0:'typical'},inplace=True)
combined_df_wtyp = combined_df.merge(typical_df,left_on='patient',right_on='Patient',how='inner')

In [ ]:
typ_df = combined_df_wtyp.copy()
typ_df['typical'] = typ_df['typical'].fillna(0)
typ_df = typ_df.dropna(subset=['outcome','typical'],axis=0)
contingency_rev = pd.crosstab(typ_df['typical'],typ_df['outcome'])
odds_ratio = contingency_rev.iloc[0,0]*contingency_rev.iloc[1,1]/(contingency_rev.iloc[0,1]*contingency_rev.iloc[1,0])

# Chi-squared test
chi2_rev, p_rev, = sc.stats.fisher_exact(contingency_rev)
print(chi2_rev, p_rev, odds_ratio)
cramers_v_rev = association(contingency_rev.values)

# True: mtle (target), Predicted: stim_sz (used as predictor)
cm = confusion_matrix(typ_df['outcome'],typ_df['typical'])  # rows = true mtle, cols = stim_sz

# Extract counts
TN, FP, FN, TP = cm.ravel()
print(cm)
# Compute PPV and NPV
ppv = TP / (TP + FP) if (TP + FP) else float('nan')
npv = TN / (TN + FN) if (TN + FN) else float('nan')

print(f"PPV (P(MTLE=1 | stim_sz=1)): {ppv:.3f}")
print(f"NPV (P(MTLE=0 | stim_sz=0)): {npv:.3f}")
print("Contingency Table (P(mtle | stim_sz)):\n", contingency_rev)
print(f"Chi2 = {chi2_rev:.4f}, p = {p_rev:.4f}, Cramer's V = {cramers_v_rev:.4f}")

9.333333333333334 0.11888111888111887 9.333333333333334
[[4 3]
 [1 7]]
PPV (P(MTLE=1 | stim_sz=1)): 0.700
NPV (P(MTLE=0 | stim_sz=0)): 0.800
Contingency Table (P(mtle | stim_sz)):
 outcome  0.0  1.0
typical          
0          4    1
1          3    7
Chi2 = 9.3333, p = 0.1189, Cramer's V = 0.4725


In [81]:
typical_df = sz_metadata.groupby('Patient').apply(lambda x: np.isin(1,x.typical).astype(int).item()).reset_index()
typical_df.rename(columns={0:'typical'},inplace=True)
combined_df_wtyp = combined_df.merge(typical_df,left_on='patient',right_on='Patient',how='left')

In [82]:
typ_df = combined_df_wtyp.copy()
typ_df['typical'] = typ_df['typical'].fillna(0)
typ_df = typ_df.dropna(subset=['outcome','typical'],axis=0)
contingency_rev = pd.crosstab(typ_df['typical'],typ_df['outcome'])
odds_ratio = contingency_rev.iloc[0,0]*contingency_rev.iloc[1,1]/(contingency_rev.iloc[0,1]*contingency_rev.iloc[1,0])

# Chi-squared test
chi2_rev, p_rev, dof_rev, expected_rev = chi2_contingency(contingency_rev)
print(chi2_rev, p_rev, dof_rev, expected_rev, odds_ratio)
cramers_v_rev = association(contingency_rev.values)

# True: mtle (target), Predicted: stim_sz (used as predictor)
cm = confusion_matrix(typ_df['outcome'],typ_df['typical'])  # rows = true mtle, cols = stim_sz

# Extract counts
TN, FP, FN, TP = cm.ravel()
print(cm)
# Compute PPV and NPV
ppv = TP / (TP + FP) if (TP + FP) else float('nan')
npv = TN / (TN + FN) if (TN + FN) else float('nan')

print(f"PPV (P(MTLE=1 | stim_sz=1)): {ppv:.3f}")
print(f"NPV (P(MTLE=0 | stim_sz=0)): {npv:.3f}")
print("Contingency Table (P(mtle | stim_sz)):\n", contingency_rev)
print(f"Chi2 = {chi2_rev:.4f}, p = {p_rev:.4f}, Cramer's V = {cramers_v_rev:.4f}")

0.13392857142857142 0.7143930376343263 1 [[14. 21.]
 [ 4.  6.]] 1.75
[[15  3]
 [20  7]]
PPV (P(MTLE=1 | stim_sz=1)): 0.700
NPV (P(MTLE=0 | stim_sz=0)): 0.429
Contingency Table (P(mtle | stim_sz)):
 outcome  0.0  1.0
typical          
0.0       15   20
1.0        3    7
Chi2 = 0.1339, p = 0.7144, Cramer's V = 0.1091


In [19]:
df.shape[0]

15